In [1]:
import numpy as np
import os

# Variables globales:

letras = ['A','B','D','E','F','G','H','I','J','K','L','M','N','Ñ','O','P','Q','R','S','T','U','V','W','X','Y','Z']
estilo_tablero = {'punto':' · ', 'agua':' = ', 'barco':' O ','tocado':' X '}

In [2]:
class Jugador:
    def __init__(self, name, ndim = 10, tipo = 'jugador'):
        self.name = name
        self.tipo = tipo
        self.disparos = []

        tablero = np.full((2,ndim,ndim),estilo_tablero['punto'])
        self.flota, self.radar = tablero[0], tablero[1]
        self.ndim = ndim
        self.letras = letras[:ndim]

    def mostrar_tablero(self):
        ndim = len(self.flota)
        print()
        for fila in range(ndim):
            if fila == 0:
                print(end = '  ')
                for n in range(2):
                    [print(' ', n, end = ' ') for n in range(1,self.ndim + 1)]
                    print('\t', end = '  ')
            print('\n',letras[fila], end = ' ')
            [print(posicion, end = ' ') for posicion in self.flota[fila]]
            print('\t', letras[fila], end = ' ')
            [print(posicion, end = ' ') for posicion in self.radar[fila]]
        print()

    def check_barcos(self):
        if np.any(self.flota == estilo_tablero['barco']):
            return True
        else:
            return False
    
    def activar_radar(self, flota_enemigo):
        self.radar = np.where(flota_enemigo == estilo_tablero['barco'], estilo_tablero['punto'], flota_enemigo)
        return self.radar

**Funciones de disparo y limpieza de pantalla**

In [26]:
def recibir_disparo(tablero, coordenada):
    if tablero[coordenada] == estilo_tablero['barco']:
        tablero[coordenada] = estilo_tablero['tocado']
        return True
    elif tablero[coordenada] == estilo_tablero['tocado']:
        print("Ya has disparado aquí")
        return False
    else:
        tablero[coordenada] = estilo_tablero['agua']
        print("Agua")
        return False

def disparo_coordenadas(tablero, coordenada):    
    fila, columna = coordenada[0], coordenada[1:]
    if len(coordenada) > 3:
        return False
    elif columna.isnumeric() != True:
        return False
    elif fila.upper() not in tablero.letras or int(columna) > tablero.ndim:
        return False
    fila = letras.index(fila.upper())
    columna = int(columna)-1
    return (fila,columna)

def disparo_random(tablero):
    coordenada = (np.random.randint(0,len(tablero)-1),np.random.randint(0,len(tablero)-1))
    while tablero[coordenada] == 'X' or tablero[coordenada] == '~':
        coordenada = (np.random.randint(0,len(tablero)-1),np.random.randint(0,len(tablero)-1))
    recibir_disparo(tablero,coordenada)

def clear(): # Limpiar pantalla
    if os.name == "posix":
        os.system ("clear")
    elif os.name == ("ce", "nt", "dos"):
        os.system ("cls")

**Función de turnos de batalla**

In [23]:
def turno_batalla():
    duo = [jugador,cpu]

    while True:
        atacante, enemigo = duo[0], duo[1]

        if atacante.name == 'vision':
            print(f'Tablero {cpu.name}')
            enemigo.activar_radar(jugador.flota)
            enemigo.mostrar_tablero()

        if atacante.tipo == 'jugador':
            print(f'Turno {atacante.name} \t\t CAÑONAZOS: {atacante.disparos.count(True)}/{len(atacante.disparos)}')
            jugador.activar_radar(cpu.flota)
            jugador.mostrar_tablero()
            
            coordenada = input('Introduce coordenada de disparo: ')
            disparo = disparo_coordenadas(enemigo, coordenada)
            if coordenada == 'salir':
                break
            while disparo == False:
                coordenada = input('Coordenadas incorrectas. Vuelve a intentarlo: ')
                disparo = disparo_coordenadas(enemigo, coordenada)
            acierto = recibir_disparo(enemigo.flota, disparo)
            atacante.disparos.append(acierto)
        else:
            print(f'Turno {atacante.name} \t\t CAÑONAZOS: {atacante.disparos.count(True)}/{len(atacante.disparos)}')
            acierto = disparo_random(enemigo.flota) 
            atacante.disparos.append(acierto)
        
        if acierto == True:
            if enemigo.check_barcos() != True:
                print('Ganaste')
                break
        else:
            duo.reverse()
            continue
    
        clear()    # Limpiamos la pantalla (no funciona en notebooks)
    
    print('Salida -> Función de menú')
    #quit()

**Prueba:**

In [25]:
jugador = Jugador('Victor')
cpu = Jugador('cpu', tipo = 'cpu')

barcos = [(1,4),(2,4),(3,4),(5,6),(5,7),(5,8)]
disparos = [(1,0), (1,1), (7,4), (7,5), (7,6), (7,7)]
for disparo,barco in zip(disparos,barcos):
    jugador.flota[barco] = estilo_tablero['barco']
    cpu.flota[disparo] = estilo_tablero['barco']

turno_batalla()

Turno Victor 		 CAÑONAZOS: 0/0

    1   2   3   4   5   6   7   8   9   10 	    1   2   3   4   5   6   7   8   9   10 	  
 A  ·   ·   ·   ·   ·   ·   ·   ·   ·   ·  	 A  ·   ·   ·   ·   ·   ·   ·   ·   ·   ·  
 B  ·   ·   ·   ·   O   ·   ·   ·   ·   ·  	 B  ·   ·   ·   ·   ·   ·   ·   ·   ·   ·  
 D  ·   ·   ·   ·   O   ·   ·   ·   ·   ·  	 D  ·   ·   ·   ·   ·   ·   ·   ·   ·   ·  
 E  ·   ·   ·   ·   O   ·   ·   ·   ·   ·  	 E  ·   ·   ·   ·   ·   ·   ·   ·   ·   ·  
 F  ·   ·   ·   ·   ·   ·   ·   ·   ·   ·  	 F  ·   ·   ·   ·   ·   ·   ·   ·   ·   ·  
 G  ·   ·   ·   ·   ·   ·   O   O   O   ·  	 G  ·   ·   ·   ·   ·   ·   ·   ·   ·   ·  
 H  ·   ·   ·   ·   ·   ·   ·   ·   ·   ·  	 H  ·   ·   ·   ·   ·   ·   ·   ·   ·   ·  
 I  ·   ·   ·   ·   ·   ·   ·   ·   ·   ·  	 I  ·   ·   ·   ·   ·   ·   ·   ·   ·   ·  
 J  ·   ·   ·   ·   ·   ·   ·   ·   ·   ·  	 J  ·   ·   ·   ·   ·   ·   ·   ·   ·   ·  
 K  ·   ·   ·   ·   ·   ·   ·   ·   ·   ·  	 K  ·   ·   ·   ·   ·   ·   ·   ·   ·   ·